In [1]:
import numpy as np
import pandas as pd
import warnings
from fractions import Fraction
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', None)
from xml.dom import minidom
import xml.etree.ElementTree as ET

In [2]:
l=['f73-8-2018-987592-possessions.xml','f73-8-2018-987593-possessions.xml','f73-8-2018-987594-possessions.xml','f73-8-2018-987595-possessions.xml',
   'f73-8-2018-987596-possessions.xml','f73-8-2018-987597-possessions.xml','f73-8-2018-987598-possessions.xml','f73-8-2018-987599-possessions.xml',
  'f73-8-2018-987600-possessions.xml','f73-8-2018-987601-possessions.xml','f73-8-2018-987602-possessions.xml','f73-8-2018-987603-possessions.xml',
  'f73-8-2018-987604-possessions.xml','f73-8-2018-987605-possessions.xml','f73-8-2018-987606-possessions.xml','f73-8-2018-987607-possessions.xml',
  'f73-8-2018-987608-possessions.xml','f73-8-2018-987609-possessions.xml','f73-8-2018-987610-possessions.xml','f73-8-2018-987611-possessions.xml',
  'f73-8-2018-987612-possessions.xml','f73-8-2018-987613-possessions.xml','f73-8-2018-987614-possessions.xml','f73-8-2018-987615-possessions.xml',
  'f73-8-2018-987616-possessions.xml','f73-8-2018-987617-possessions.xml','f73-8-2018-987618-possessions.xml','f73-8-2018-987619-possessions.xml',
    'f73-8-2018-987620-possessions.xml','f73-8-2018-987621-possessions.xml','f73-8-2018-987622-possessions.xml','f73-8-2018-987623-possessions.xml',
   'f73-8-2018-987624-possessions.xml','f73-8-2018-987625-possessions.xml','f73-8-2018-987626-possessions.xml','f73-8-2018-987627-possessions.xml',
   'f73-8-2018-987628-possessions.xml','f73-8-2018-987629-possessions.xml','f73-8-2018-987630-possessions.xml','f73-8-2018-987631-possessions.xml']
centerbacks=pd.DataFrame()
for p in l:
    poss = ET.parse(p)
    root = poss.getroot()
    game_info={}
    for child in root:
        #print(child.tag, child.attrib)
        game_info=child.attrib
        Q=[]
        for c in child: 
            for f in c:
                f.attrib["event_number"]=c.attrib['id']
                f.attrib["event_id"]=c.attrib['event_id']
                f.attrib["type_id"]=c.attrib['type_id']
                f.attrib["period_id"]=c.attrib['period_id']
                f.attrib["min"]=c.attrib['min']
                f.attrib["sec"]=c.attrib['sec']
                f.attrib["team_id"]=c.attrib['team_id']
                if "player_id" in c.attrib.keys():
                    f.attrib["player_id"]=c.attrib["player_id"]
                else:
                    f.attrib["player_id"]="-1"

                f.attrib["outcome"]=c.attrib['outcome']
                f.attrib["x"]=c.attrib['x']
                f.attrib["y"]=c.attrib['y']
                Q.append(f.attrib)
    Qs=pd.DataFrame(Q)
    Qs
    homegoals=len(Qs[(Qs.type_id=="16")&(Qs.team_id==game_info['home_team_id'])].groupby(Qs.event_number))
    awaygoals=len(Qs[(Qs.type_id=="16")&(Qs.team_id==game_info['away_team_id'])].groupby(Qs.event_number))
    defenders=[]
    for n in Qs[Qs['qualifier_id']=='30'].index:
        defenders.append(Qs[Qs['qualifier_id']=='30']['value'][n].split(",")[4].strip())
        defenders.append(Qs[Qs['qualifier_id']=='30']['value'][n].split(",")[5].strip())
    defenders
    myset = set(defenders)
    distinct_defs=list(myset)
    #print(distinct_defs)
    cb_df=pd.DataFrame()
    for m in distinct_defs:
        cb_df=cb_df.append(Qs[Qs.player_id==m])

    cenbacks=pd.DataFrame(columns=('game_id','team_id','player_id','total_passes',
                               'successful passes','pass completion rate',
                               'total aerial duels','aerial won','aerial win rate',
                              'total ground duels','ground duels won', 'ground duels win rate',
                              'fouls conceded','handball conceded','interceptions','tackles won',
                               'tackles lost-challenge','clearances','goals conceded'))
    for i in distinct_defs:

        ## Game id ##
        gameid=int(game_info['id'])

        ## Team they play for ##
        team=cb_df[cb_df.player_id==i]['team_id'][0:1].values[0]
        playerid=i

        ## Total passes ##
        allpasses=cb_df[(cb_df.player_id==i) & (cb_df.type_id=='1')]
        for j in allpasses:
            exceptions={}
            exceptions=allpasses[(allpasses.qualifier_id=='2') | (allpasses.qualifier_id=='107') | (allpasses.qualifier_id=='123')]['event_number']
        for k in exceptions.values:
            allpasses=allpasses[allpasses.event_number!=k]
        totalpasses=len(allpasses.groupby(allpasses.event_number))

        ## successful passes ##
        successpass=cb_df[(cb_df.player_id==i) & (cb_df.type_id=='1') & (cb_df.outcome=='1')]
        for j in successpass:
            exceptions={}
            exceptions=successpass[(successpass.qualifier_id=='2') | (successpass.qualifier_id=='107') | (successpass.qualifier_id=='123')]['event_number']
        for k in exceptions.values:
            successpass=successpass[successpass.event_number!=k]
        succpasses=len(successpass.groupby(successpass.event_number))

        ## pass completion rate ##
        if totalpasses!=0:
            passcomprate=round(succpasses/totalpasses,2)
        else:
            passcomprate=-1
            
        ## total aerial duel ##
        allaerial=cb_df[((cb_df.player_id==i)) & ((cb_df.type_id=='44') | ((cb_df.type_id=='4')&((cb_df.qualifier_id=='264'))))]
        totalaerial=len(allaerial.groupby(allaerial.event_number))

        ## aerial won ##
        wonaerial=cb_df[((cb_df.player_id==i)) & ((cb_df.type_id=='44') | ((cb_df.type_id=='4')&((cb_df.qualifier_id=='264')))) &(cb_df.outcome=='1')]
        aerial_won=len(wonaerial.groupby(wonaerial.event_number))

        ## aerial win rate ##
        if totalaerial!=0:
            aerialwr=round(aerial_won/totalaerial,2)
        else:
            aerialwr=-1

        ## total ground duels ##
        totalground=cb_df[((cb_df.player_id==i)) & ( (cb_df.type_id=='3') | ((cb_df.type_id=='4')&((cb_df.qualifier_id!='264'))) |(cb_df.type_id=='7')|(cb_df.type_id=='45')|(cb_df.type_id=='50')|(cb_df.type_id=='54'))]
        totalgrounduels=len(totalground.groupby(totalground.event_number))

        ## ground duels won ##
        wonground=cb_df[((cb_df.player_id==i)) & ( (cb_df.type_id=='3') | ((cb_df.type_id=='4')&((cb_df.qualifier_id!='264'))) |(cb_df.type_id=='7')|(cb_df.type_id=='54'))]
        ground_won=len(wonground.groupby(wonground.event_number))

        ## ground duels win rate ##
        if totalgrounduels!=0:
            groundwr=round(ground_won/totalgrounduels,2)
        else:
            groundwr=-1
            

        ## fouls conceded ##
        fouls=cb_df[((cb_df.player_id==i)) & ((cb_df.type_id=='4')&((cb_df.outcome=='0')))]
        foulsconceded=len(fouls.groupby(fouls.event_number))

        ## handball conceded ##
        handball=cb_df[(cb_df.player_id==i) & ((cb_df.type_id=='4')&(cb_df.outcome=='0')&(cb_df.qualifier_id=='10'))]
        handballconceded=len(handball.groupby(handball.event_number))

        ## interceptions ##
        intercept=cb_df[(cb_df.player_id==i) & (cb_df.type_id=='8')]
        interceptions=len(intercept.groupby(intercept.event_number))

        ## tackles won ##
        tackles=cb_df[(cb_df.player_id==i) & (cb_df.type_id=='7')]
        wontackles=len(tackles.groupby(tackles.event_number))

        ## tackles lost ##
        tackleslost=cb_df[(cb_df.player_id==i) & ((cb_df.type_id=='45')&(cb_df.outcome=='0'))]
        losttackles=len(tackleslost.groupby(tackleslost.event_number))

        ## clearances ##
        clear=cb_df[(cb_df.player_id==i) & (cb_df.type_id=='12')]
        clearances=len(clear.groupby(clear.event_number))

        ## goals team conceded ##
        if cb_df[cb_df.player_id==i]['team_id'][0:1].values[0]==game_info['home_team_id']:
            goalsconc=awaygoals
        elif cb_df[cb_df.player_id==i]['team_id'][0:1].values[0]==game_info['away_team_id']:
            goalsconc=homegoals



        cenbacks.loc[i] = [gameid,team,playerid,totalpasses,succpasses,passcomprate,totalaerial,aerial_won,aerialwr,
                           totalgrounduels,ground_won,groundwr,foulsconceded,handballconceded,interceptions,wontackles,
                           losttackles,clearances,goalsconc]

    cenbacks 
    centerbacks=centerbacks.append(cenbacks)
centerbacks

,game_id,team_id,player_id,total_passes,successful passes,pass completion rate,total aerial duels,aerial won,aerial win rate,total ground duels,ground duels won,ground duels win rate,fouls conceded,handball conceded,interceptions,tackles won,tackles lost-challenge,clearances,goals conceded
56917,987592,91,56917,66,50,0.76,14,7,0.5,6,5,0.83,3,0,2,1,1,11,0
49982,987592,97,49982,27,18,0.67,10,6,0.6,2,1,0.5,0,0,2,1,1,8,2
49382,987592,97,49382,19,14,0.74,2,2,1.0,5,5,1.0,1,0,1,4,0,8,2
126184,987592,91,126184,59,56,0.95,4,1,0.25,2,1,0.5,0,0,0,1,1,8,0
97299,987593,43,97299,72,67,0.93,3,3,1.0,3,3,1.0,1,0,1,2,0,1,0
39476,987593,3,39476,37,30,0.81,1,1,1.0,3,3,1.0,2,0,0,1,0,9,2
146941,987593,43,146941,59,55,0.93,1,0,0.0,4,3,0.75,2,0,2,0,1,5,0
69140,987593,3,69140,40,36,0.90,2,2,1.0,6,5,0.83,1,0,1,4,1,5,2
49413,987594,31,49413,35,31,0.89,5,5,1.0,2,1,0.5,0,0,1,1,1,11,0
61739,987594,54,61739,84,74,0.88,6,4,0.67,5,5,1.0,1,0,1,3,0,6,2


In [191]:
## Number of unique Centerbacks 
len(centerbacks.groupby('player_id'))

62

In [3]:
## Get the names of CBs ##

l=['srml-8-2018-f987592-matchresults.xml','srml-8-2018-f987593-matchresults.xml','srml-8-2018-f987594-matchresults.xml','srml-8-2018-f987595-matchresults.xml',
   'srml-8-2018-f987596-matchresults.xml','srml-8-2018-f987597-matchresults.xml','srml-8-2018-f987598-matchresults.xml','srml-8-2018-f987599-matchresults.xml',
  'srml-8-2018-f987600-matchresults.xml','srml-8-2018-f987601-matchresults.xml','srml-8-2018-f987602-matchresults.xml','srml-8-2018-f987603-matchresults.xml',
   'srml-8-2018-f987604-matchresults.xml','srml-8-2018-f987605-matchresults.xml','srml-8-2018-f987606-matchresults.xml','srml-8-2018-f987607-matchresults.xml',
  'srml-8-2018-f987608-matchresults.xml','srml-8-2018-f987609-matchresults.xml','srml-8-2018-f987610-matchresults.xml','srml-8-2018-f987611-matchresults.xml',
   'srml-8-2018-f987612-matchresults.xml','srml-8-2018-f987613-matchresults.xml','srml-8-2018-f987614-matchresults.xml','srml-8-2018-f987615-matchresults.xml',
   'srml-8-2018-f987616-matchresults.xml','srml-8-2018-f987617-matchresults.xml','srml-8-2018-f987618-matchresults.xml','srml-8-2018-f987619-matchresults.xml',
   'srml-8-2018-f987620-matchresults.xml','srml-8-2018-f987621-matchresults.xml','srml-8-2018-f987622-matchresults.xml','srml-8-2018-f987623-matchresults.xml',
  'srml-8-2018-f987624-matchresults.xml','srml-8-2018-f987625-matchresults.xml','srml-8-2018-f987626-matchresults.xml','srml-8-2018-f987627-matchresults.xml',
   'srml-8-2018-f987628-matchresults.xml','srml-8-2018-f987629-matchresults.xml','srml-8-2018-f987630-matchresults.xml','srml-8-2018-f987631-matchresults.xml']



namesdf=pd.DataFrame(columns=('player_id','First Name','Last Name'))
for p in l:
    mresult = ET.parse(p)
    root2 = mresult.getroot()
    for child in root2:
        gameid=int(child.attrib['uID'][1:])

    gamecbs=centerbacks.loc[centerbacks.game_id==gameid]



    for i in gamecbs.index:
        
        player="p"+i
        s=gamecbs.loc[i]['team_id']
        team="t"+s
        if type(team)==pd.core.series.Series:
            team=team[0]
        elif type(team)==str:
            team=team

        for child in root2:
            for c in child:
                for f in c:
                    if f.attrib.get('uID')==player:
                        for g in f:
                            for h in g:
                                if h.tag=='First':
                                    name=h.text
                                elif h.tag=='Last':
                                    last=h.text
        namesdf.loc[i]=[i,name,last]
names=[]
lastnames=[]
for i in centerbacks.index:
    names.append(namesdf.loc[i]['First Name'])
    lastnames.append(namesdf.loc[i]['Last Name'])
    
centerbacks['last name']=lastnames
centerbacks['first name']=names

centerbacks

,game_id,team_id,player_id,total_passes,successful passes,pass completion rate,total aerial duels,aerial won,aerial win rate,total ground duels,...,ground duels win rate,fouls conceded,handball conceded,interceptions,tackles won,tackles lost-challenge,clearances,goals conceded,last name,first name
56917,987592,91,56917,66,50,0.76,14,7,0.5,6,...,0.83,3,0,2,1,1,11,0,Cook,Steve
49982,987592,97,49982,27,18,0.67,10,6,0.6,2,...,0.5,0,0,2,1,1,8,2,Morrison,Sean
49382,987592,97,49382,19,14,0.74,2,2,1.0,5,...,1.0,1,0,1,4,0,8,2,Ecuele Manga,Bruno
126184,987592,91,126184,59,56,0.95,4,1,0.25,2,...,0.5,0,0,0,1,1,8,0,Aké,Nathan
97299,987593,43,97299,72,67,0.93,3,3,1.0,3,...,1.0,1,0,1,2,0,1,0,Stones,John
39476,987593,3,39476,37,30,0.81,1,1,1.0,3,...,1.0,2,0,0,1,0,9,2,Papastathopoulos,Sokratis
146941,987593,43,146941,59,55,0.93,1,0,0.0,4,...,0.75,2,0,2,0,1,5,0,Laporte,Aymeric
69140,987593,3,69140,40,36,0.90,2,2,1.0,6,...,0.83,1,0,1,4,1,5,2,Mustafi,Shkodran
49413,987594,31,49413,35,31,0.89,5,5,1.0,2,...,0.5,0,0,1,1,1,11,0,Tomkins,James
61739,987594,54,61739,84,74,0.88,6,4,0.67,5,...,1.0,1,0,1,3,0,6,2,Le Marchand,Maxime


In [4]:
centerbacks.to_excel('Centerbacks_from_xml.xlsx')